# Client IP

In [ ]:
!curl ipecho.net/plain

# TMIDIX Python module

In [ ]:
!git clone https://github.com/asigalov61/tegridy-tools

In [ ]:
%cd /content/tegridy-tools/tegridy-tools

import TMIDIX

%cd /content/

# Load input MIDI file

In [ ]:
f = '/content/tegridy-tools/tegridy-tools/seed2.mid'

#=======================================================
# START PROCESSING

# Convering MIDI to ms score with MIDI.py module
score = TMIDIX.midi2ms_score(open(f, 'rb').read())

# INSTRUMENTS CONVERSION CYCLE
events_matrix = []
melody_chords_f = []
melody_chords_f1 = []

itrack = 1

patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

patch_map = [
            [0, 1, 2, 3, 4, 5, 6, 7], # Piano 
            [24, 25, 26, 27, 28, 29, 30], # Guitar
            [32, 33, 34, 35, 36, 37, 38, 39], # Bass
            [40, 41], # Violin
            [42, 43], # Cello
            [46], # Harp
            [56, 57, 58, 59, 60], # Trumpet
            [64, 65, 66, 67, 68, 69, 70, 71], # Sax
            [72, 73, 74, 75, 76, 77, 78], # Flute
            [-1], # Drums
            [52, 53], # Choir
            [16, 17, 18, 19, 20] # Organ
            ]

while itrack < len(score):
  for event in score[itrack]:         
      if event[0] == 'note' or event[0] == 'patch_change':
          events_matrix.append(event)
  itrack += 1

events_matrix.sort(key=lambda x: x[1])

events_matrix1 = []

for event in events_matrix:
  if event[0] == 'patch_change':
      patches[event[2]] = event[3]

  if event[0] == 'note':
      event.extend([patches[event[3]]])
      once = False

      for p in patch_map:
          if event[6] in p and event[3] != 9: # Except the drums
              event[3] = patch_map.index(p)
              once = True

      if not once and event[3] != 9: # Except the drums
          event[3] = 15 # All other instruments/patches channel
          event[5] = max(80, event[5])

      if event[3] < 12: # We won't write chans 12-16 for now...
          events_matrix1.append(event)

#=======================================================
# PRE-PROCESSING

# checking number of instruments in a composition
instruments_list_without_drums = list(set([y[3] for y in events_matrix1 if y[3] != 9]))

if len(events_matrix1) > 0 and len(instruments_list_without_drums) > 0:

  # recalculating timings
  for e in events_matrix1:
      e[1] = int(e[1] / 8) # Max 1 seconds for start-times
      e[2] = int(e[2] / 30) # Max 2 seconds for durations

  # Sorting by pitch, then by start-time
  events_matrix1.sort(key=lambda x: x[4], reverse=True)
  events_matrix1.sort(key=lambda x: x[1])

  #=======================================================
  # FINAL PRE-PROCESSING

  melody_chords = []

  pe = events_matrix1[0]

  for e in events_matrix1:
    if e[1] >= 0 and e[2] > 0:

      # Cliping all values...
      tim = max(0, min(255, e[1]-pe[1]))             
      dur = max(1, min(127, e[2]))
      cha = max(0, min(11, e[3]))
      ptc = max(1, min(127, e[4]))
      vel = max(1, min(127, e[5]))

      # Writing final note 
      melody_chords.append([tim, dur, cha, ptc, vel])

      pe = e

#=======================================================
# Velocities map
#=======================================================

# Default fixed velocities for each channel/instrument
velocities_map = [80, 80, 70, 100, 90, 80, 100, 100, 100, 90, 110, 100]

# Extracting velocities from the MIDI file
for i in range(12):
  vels = [m[4] for m in melody_chords if m[2] == i]

  avg_vel = 0

  if len(vels) > 0:
    avg_vel = int(sum(vels) / len(vels))

  if avg_vel > 20:
    velocities_map[i] = avg_vel

#=======================================================
# MAIN PROCESSING CYCLE
#=======================================================

for m in melody_chords:

  # WRITING EACH NOTE HERE
  time = m[0]
  cha_dur = (m[2] * 128) + m[1]
  cha_ptc = (m[2] * 128) + m[3]
    
  melody_chords_f.extend([time, cha_dur+256, cha_ptc+1792])

print(melody_chords_f)

# API requests examples

## Curl request with improv tokens sequence

In [ ]:
!curl --header "Content-Type: application/json" --request POST --data '{"input_notes":[[3343, 3330, 3339]],"notes_count":10}' http://152.70.113.241:6000/

## Python request with the custom MIDI above

In [ ]:
import requests
import json

input_notes = melody_chords_f[:240] # Prime sequence (of 240 / 3 == 80 notes)

number_of_notes_to_generate = 128

data = json.dumps({
		
				"input_notes": input_notes, 

				"notes_count": number_of_notes_to_generate, # Number of notes to generate

		    })

headers = {"Content-Type": "application/json"}

print('=' * 90)
print('Requesting data... Please wait...')
response = requests.post('http://152.70.113.241:6000', headers=headers, data=data)
print('=' * 90)
print('Response received! :)')
print('=' * 90)
res = response.json()
print('=' * 90)
print(res)
print('=' * 90)
print(res['input_notes'])
print(res['output_notes'])
print('=' * 90)

## Convert API response to MIDI

In [ ]:
print('=' * 70)

out1 = input_notes + res['output_notes'] # Input notes + API response

print('Sample INTs', out1[:12])
print('=' * 70)

if len(out1) != 0:
  
    song = out1
    song_f = []

    time = 0
    dur = 0
    channel = 0
    pitch = 0
    vel = 90

    for ss in song:

      if ss > 0 and ss < 256:

          time += ss * 8
        
      if ss >= 256 and ss < 256+(12*128):

          dur = ((ss-256) % 128) * 30
          
      if ss >= 256+(12*128) and ss < 256+(12*128)+(12*128):
          channel = (ss-(256+(12*128))) // 128
          pitch = (ss-(256+(12*128))) % 128
          vel = velocities_map[channel]

          song_f.append(['note', time, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Euterpe X',  
                                                        output_file_name = '/content/Euterpe-X-Music-Composition_'+str(i), 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)


    print('=' * 70)